# Generating polyphony

last time, the generated music was monophonic, this time, it'll be polyphonic

In [1]:
import math
import os
import time

import magenta.music as mm
import tensorflow as tf
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.music import DEFAULT_QUARTERS_PER_MINUTE
from magenta.protobuf.generator_pb2 import GeneratorOptions
from magenta.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter

In [2]:
from magenta.models.polyphony_rnn import polyphony_sequence_generator

In [3]:
from magenta.models.shared import sequence_generator_bundle 

Just as last time, starting with downloading the right bundle file

In [4]:
bundle_name = "polyphony_rnn.mag"

In [5]:
mm.notebook_utils.download_bundle(bundle_name, "bundles")
bundle = sequence_generator_bundle.read_bundle_file(
    os.path.join("bundles", bundle_name))

In [6]:
for key, value in polyphony_sequence_generator.get_generator_map().items():
    print (key)

polyphony


For the polyphony rnn, there is only one option

In [12]:
#initialize the model
generator_map = polyphony_sequence_generator.get_generator_map()
generator = generator_map['polyphony'](checkpoint=None, bundle=bundle)

In [13]:
type(generator)

magenta.models.polyphony_rnn.polyphony_sequence_generator.PolyphonyRnnSequenceGenerator

In [8]:
generator.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\alecr\AppData\Local\Temp\tmpubjjb_d1\model.ckpt


For this, we won't use a specific primer, just an empty NoteSequence

In [9]:
primer_sequence = NoteSequence()

This is empty, and since it's empty, I'll just go ahead and use the same basic tempo everything does, which is the same as the default quarters per minute

In [12]:
qpm = DEFAULT_QUARTERS_PER_MINUTE

In [13]:
seconds_per_step = 60.0 / qpm / getattr(generator, "steps_per_quarter", 4)

In [14]:
primer_sequence_length_steps = math.ceil(primer_sequence.total_time
                                           / seconds_per_step)
primer_sequence_length_time = primer_sequence_length_steps * seconds_per_step

In [15]:
primer_end_adjust = (0.00001 if primer_sequence_length_time > 0 else 0)
primer_start_time = 0
primer_end_time = (primer_start_time
                     + primer_sequence_length_time
                     - primer_end_adjust)

In [16]:
total_length_steps = 512

In [17]:
generation_length_steps = total_length_steps - primer_sequence_length_steps


In [18]:
if generation_length_steps <= 0:
    raise Exception("Total length in steps too small "
                    + "(" + str(total_length_steps) + ")"
                    + ", needs to be at least one bar bigger than primer "
                    + "(" + str(primer_sequence_length_steps) + ")")
generation_length_time = generation_length_steps * seconds_per_step

In [19]:
generation_start_time = primer_end_time
generation_end_time = (generation_start_time
                         + generation_length_time
                         + primer_end_adjust)

In [20]:
print(f"Primer time: [{primer_start_time}, {primer_end_time}]")
print(f"Generation time: [{generation_start_time}, {generation_end_time}]")

Primer time: [0, 0.0]
Generation time: [0.0, 64.0]


In [21]:
generator_options = GeneratorOptions()

these are the same as in the monophonic rnn, but there's additional parameters here

In [22]:
temperature: float = 1.0
beam_size: int = 1
branch_factor: int = 1
steps_per_iteration: int = 1

In [23]:
generator_options.args['temperature'].float_value = temperature
generator_options.args['beam_size'].int_value = beam_size
generator_options.args['branch_factor'].int_value = branch_factor
generator_options.args['steps_per_iteration'].int_value = steps_per_iteration

In [24]:
generator_options.args['condition_on_primer'].bool_value = True #this is because of ...

In [25]:
inject_primer_during_generation=False

In [26]:
generator_options.args['no_inject_primer_during_generation'].bool_value = (
    not inject_primer_during_generation)

In [27]:
generator_options.generate_sections.add(
    start_time=generation_start_time,
    end_time=generation_end_time)

end_time: 64.0

In [28]:
for key, value in generator_options.args.items():
    print(key)

temperature
beam_size
branch_factor
steps_per_iteration
condition_on_primer
no_inject_primer_during_generation


The difference between condition_on_primer and no_inject_primer_during_generation is that the first says whether to use a primer to generate the sequence, and the latter is on whether to incorporate the primer as part of the output sequence

In [29]:
sequence = generator.generate(primer_sequence, generator_options)

INFO:tensorflow:Need to generate 511 more steps for this sequence, will try asking for 2555 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -931.400452 
INFO:tensorflow:Need to generate 53 more steps for this sequence, will try asking for 265 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -1050.020508 


In [30]:
midi_path = r"random_polyphony.mid"

In [31]:
mm.midi_io.note_sequence_to_midi_file(sequence=sequence, output_file=midi_path)

Looking and listening to the output, it looks like the time signature forced on it wasn't right, and you can hear some pretty standard Bach-type counterpoint there.